In [1]:
from create_dataset import *
import h5py
from utils import *
import sys
from scipy.io import wavfile
import pdb
from time import time
from utils import *
import glob

In [8]:
path2wavs = '/media/songbird/Gagan_external/songbird_data/WAV_files/'

In [3]:
birds = os.listdir(path2wavs)
birds

['b13r16',
 'b14r16',
 'b3g20',
 'b4g20',
 'b6r17',
 'b7r16',
 'b8r17',
 'g11y3',
 'g13k7',
 'g13p17',
 'g19r15',
 'g20r15',
 'g20y17',
 'g20y3',
 'g7r15',
 'k3r16',
 'k6r16',
 'p20r16',
 'p3r16',
 'r15s12',
 'r15y2',
 'r15y5',
 'r17s7',
 'r20s4']

In [4]:
path2bird = os.path.join(path2wavs, 'g13k7', 'SAP')

In [5]:
os.listdir(path2bird)

['2014-11-30', '2014-12-01']

In [13]:
path2data = os.path.join(path2bird, '2014-11-30','songs')

In [16]:
fls = os.listdir(path2data)
len(fls)

1951

In [17]:
birdfile = h5py.File('/mnt/Data/g13k7_test_compressed.h5', 'w')


In [22]:
day1 = birdfile.create_group('2014-11-30')

In [23]:
for k in day1.keys():
    print(k)

In [26]:
day1

<HDF5 group "/2014-11-30" (0 members)>

In [20]:
age = 100

In [30]:
for (i,f) in enumerate(fls):
    # first load the file 
    song, fs = load_wav_file(os.path.join(path2data, f))
    # down sample
    song = downsample(song, 2)
    # create spectrogram
    ims = to_image(song, 256)
    # now save ims as a dataset
    day1.create_dataset(f.split('.')[0]+'_'+str(age), data=ims)

In [66]:
birdfile.close()

# read back file

In [32]:
birdfile = h5py.File('/media/songbird/datapartition/mdgan_training_input_with_age_HDF/b7r16', 'a')

In [33]:
day = birdfile.create_group('2014-12-01')

In [34]:
path2data = os.path.join(path2bird, '2014-12-01','songs')
fls = os.listdir(path2data)
len(fls)

1440

In [35]:
for (i,f) in enumerate(fls):
    # first load the file 
    song, fs = load_wav_file(os.path.join(path2data, f))
    # down sample
    song = downsample(song, 2)
    # create spectrogram
    ims = to_image(song, 256)
    # now save ims as a dataset
    day.create_dataset(f.split('.')[0]+'_'+str(age), data=ims, compression = 'gzip', compression_opts = 9)

# explore the h5py file

In [10]:
birdfile = h5py.File('/media/songbird/datapartition/mdgan_training_input_with_age_HDF/b7r16', 'r')

In [ ]:
base_items = list(birdfile.items())
print(base_items)

In [12]:
base_items[0][0]

'2011-05-03'

In [14]:
day1 = birdfile.get(base_items[0][0])

In [ ]:
day1_items = list(day1.items())
day1_items

In [16]:
day1_items[0]

('b7r16U-f00001-1_0_2',
 <HDF5 dataset "b7r16U-f00001-1_0_2": shape (129, 168, 2), type "<f4">)

In [12]:
file1_spect = np.array(birdfile.get('/'+base_items[0][0]+'/'+day1_items[0][0]))

In [13]:
file1_spect.dtype

dtype('float32')

In [14]:
file1_spect.shape

(129, 136, 2)

In [18]:
birdfile.close()

In [39]:
def get_random_spectrogram_sample_from_bird(birdhdfpath, folder_names, Nsamps = 1):
    birdfile = h5py.File(birdhdfpath, 'r')
    base_items = list(birdfile.items())
    directory_names = [b[0] for b in base_items]
    # search for folder name and get random samples
    out = [None for k in range(len(folder_names))]
    for j,f in enumerate(folder_names):
        for i,d in enumerate(directory_names):
            if f in d:
                sequencelist = list(birdfile.get(base_items[i][0]).items())
                idx = np.random.choice(len(sequencelist),size=Nsamps,replace=False)
                out[j] = [None for m in range(Nsamps)]
                for k in range(len(idx)):
                    out[j][k] = np.array(birdfile.get('/'+d+'/'+sequencelist[idx[k]][0]))
    birdfile.close()
    return out

In [40]:
X = get_random_spectrogram_sample_from_bird('/media/songbird/datapartition/mdgan_training_input_with_age_HDF/b7r16', ['tutor'], Nsamps=10)

In [41]:
X[0][0].shape

(129, 208, 2)

# function for creating one birds compressed dataset

In [10]:
def create_bird_spectrogram_hdf(birdname, birddatapath, outpath, downsample_factor=2, nfft=256, standardize=False, compress_type='gzip', compress_idx=9):
    '''
    Creates HDF file for this bird. Each day of recording is a Group, 
    and the spectrogram of each wav file is a Dataset in a Group.
    Attributes are added for each group
    '''
    start = time()
    # create hdf file for this bird in the outpath folder
    birdfile = h5py.File(os.path.join(outpath, birdname), 'w')
    try:
        # get all folder names
        folders = os.listdir(birddatapath)
        # determine pseudo age
        ages = extract_pseudoage_from_folder_names(folders)
        if len(ages)==1:
            ages = [ages[0] for i in range(len(folders))]
        # go through each folder, load files, downsample (optional), standardize (optional) and create STFTs
        save_idx = 0
        for (k,fold) in enumerate(folders):
            # create group
            d = birdfile.create_group(fold)
            d.attrs['CLASS'] = 'STFT_with_Magnitude_and_Phase(2nd index in last dimension)'
            d.attrs['DTYPE'] = 'float32'
            d.attrs['PSEUDO_AGE'] = ages[k]
            d.attrs['nfft'] = nfft
            d.attrs['downsamplerate'] = downsample_factor
            d.attrs['standardized'] = standardize
            songs, fs, filenames = load_from_folder(birddatapath, fold)
            if songs:
                if downsample_factor:
                    songs = [downsample(i, downsample_factor) for i in songs]
                    if standardize:
                        songs = [s/np.std(s) for s in songs]
                ims = [to_image(i,nfft) for i in songs]
                for (i,im) in enumerate(ims):
                    d.create_dataset(filenames[i].split('.')[0]+'_'+str(save_idx)+'_'+str(ages[k]), data=im, \
                                     compression = compress_type, compression_opts = compress_idx)
        birdfile.close()
    except:
        birdfile.close()
    end = time()
    print('..... bird %s finished in %5d secs.....'%(birdname, end-start)) 

# go over all birds and do it!

In [9]:
path1 = '/media/songbird/Gagan_external/songbird_data/WAV_files/'
out_path = '/mnt/Data/mdgan_training_input_with_age_HDF/'
birds = os.listdir(path1)
birds

['b13r16',
 'b14r16',
 'b3g20',
 'b4g20',
 'b6r17',
 'b7r16',
 'b8r17',
 'g11y3',
 'g13k7',
 'g13p17',
 'g19r15',
 'g20r15',
 'g20y17',
 'g20y3',
 'g7r15',
 'k3r16',
 'k6r16',
 'p20r16',
 'p3r16',
 'r15s12',
 'r15y2',
 'r15y5',
 'r17s7',
 'r20s4']

# exclude birds if needed

In [10]:
birds = birds[7:]
birds

['g11y3',
 'g13k7',
 'g13p17',
 'g19r15',
 'g20r15',
 'g20y17',
 'g20y3',
 'g7r15',
 'k3r16',
 'k6r16',
 'p20r16',
 'p3r16',
 'r15s12',
 'r15y2',
 'r15y5',
 'r17s7',
 'r20s4']

In [11]:
for b in birds:
    print('\n ..... bird is %s ....\n'%(b))
    
    create_bird_spectrogram_hdf(b, os.path.join(path1, b, 'SAP'), out_path, 2, 256)


 ..... bird is g11y3 ....

..... bird g11y3 finished in    10 secs.....

 ..... bird is g13k7 ....

..... bird g13k7 finished in    76 secs.....

 ..... bird is g13p17 ....

..... bird g13p17 finished in    51 secs.....

 ..... bird is g19r15 ....

..... bird g19r15 finished in   526 secs.....

 ..... bird is g20r15 ....

..... bird g20r15 finished in   637 secs.....

 ..... bird is g20y17 ....

..... bird g20y17 finished in  2638 secs.....

 ..... bird is g20y3 ....

..... bird g20y3 finished in  3673 secs.....

 ..... bird is g7r15 ....

..... bird g7r15 finished in  1463 secs.....

 ..... bird is k3r16 ....

..... bird k3r16 finished in  2226 secs.....

 ..... bird is k6r16 ....

..... bird k6r16 finished in  2583 secs.....

 ..... bird is p20r16 ....

..... bird p20r16 finished in  2202 secs.....

 ..... bird is p3r16 ....

..... bird p3r16 finished in  2327 secs.....

 ..... bird is r15s12 ....

..... bird r15s12 finished in  2533 secs.....

 ..... bird is r15y2 ....

..... bird 

# make ID lists of data

In [11]:
import pdb

In [2]:
def make_IDs(birdhdfpath, birdname, id_list, age_weight_list, cnt):
    birdfile = h5py.File(birdhdfpath, 'r')
    all_grps = list(birdfile.items())
    # cycle over days for this bird
    for g in all_grps:
        day = birdfile.get(g[0])
        day_wav_list = list(day.items())
        # cycle over files
        for f in day_wav_list:
            duration = np.array(f[1]).shape[1]
            age_weight = 1 - float(f[0].split('_')[-1])/100
            if age_weight==0.0:
                # for adults age is automaticaly 100, so weight 
                # be 0 which is undesirable
                age_weight = 0.1
            age_weight_list.append(age_weight)
            id_list.append({'id':cnt, 'birdname': birdname, 'filepath': birdhdfpath, \
                            'within_file': '/'+g[0]+'/'+f[0], 'age_weight': age_weight,
                           'duration':duration})
            cnt += 1
    birdfile.close()
    return id_list, age_weight_list, cnt

In [3]:
def make_ID_list(path2birds):
    birds = os.listdir(path2birds)
    id_list = []
    age_weight_list = []
    cnt = 0
    for (i,b) in enumerate(birds):
        id_list, age_weight_list, cnt = make_IDs(os.path.join(path2birds, b), b, id_list, age_weight_list, cnt)
        print('..... %d of %d birds indexed .....'%(i, len(birds)))
    return id_list, age_weight_list, cnt

In [4]:
id_list, age_weight_list, cnt = make_ID_list('/media/songbird/datapartition/mdgan_training_input_with_age_HDF/')

..... 0 of 23 birds indexed .....
..... 1 of 23 birds indexed .....
..... 2 of 23 birds indexed .....
..... 3 of 23 birds indexed .....
..... 4 of 23 birds indexed .....
..... 5 of 23 birds indexed .....
..... 6 of 23 birds indexed .....
..... 7 of 23 birds indexed .....
..... 8 of 23 birds indexed .....
..... 9 of 23 birds indexed .....
..... 10 of 23 birds indexed .....
..... 11 of 23 birds indexed .....
..... 12 of 23 birds indexed .....
..... 13 of 23 birds indexed .....
..... 14 of 23 birds indexed .....
..... 15 of 23 birds indexed .....
..... 16 of 23 birds indexed .....
..... 17 of 23 birds indexed .....
..... 18 of 23 birds indexed .....
..... 19 of 23 birds indexed .....
..... 20 of 23 birds indexed .....
..... 21 of 23 birds indexed .....
..... 22 of 23 birds indexed .....


In [23]:
cnt

1608376

In [24]:
with open('/media/songbird/datapartition/id_list_train.pkl','rb') as f:
    id_list_train = pickle.load(f)

age_weights_train = [ids['age_weight'] for ids in id_list_train]

with open('/media/songbird/datapartition/age_weights_training.pkl', 'wb') as f: 
    pickle.dump(age_weights_train, f, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
with open('/media/songbird/datapartition/id_list_test.pkl','rb') as f:
    id_list_test = pickle.load(f)

age_weights_test = [ids['age_weight'] for ids in id_list_test]

with open('/media/songbird/datapartition/age_weights_test.pkl', 'wb') as f: 
    pickle.dump(age_weights_test, f, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
id_list, age_weight_list, cnt = make_ID_list('/media/songbird/datapartition/mdgan_holdout_test/')

..... 0 of 2 birds indexed .....
..... 1 of 2 birds indexed .....


In [13]:
cnt

203714

In [14]:
len(id_list)

203714

In [15]:
import pickle

In [26]:
with open('/media/songbird/datapartition/mdgan_holdout_test/age_weights.pkl', 'wb') as f: 
    pickle.dump(age_weight_list, f, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
with open('/media/songbird/datapartition/id_list_holdout.pkl', 'wb') as f: 
    pickle.dump(id_list, f, protocol=pickle.HIGHEST_PROTOCOL)

# create training, validation and test sets

In [5]:
from random import shuffle

In [6]:
# split id_list into train, val and test
ids = [n for n in range(len(id_list))]
shuffle(ids)
train_valtest_ratio = 0.9
#val_test_ratio = 0.2


idstrain = ids[:round(train_valtest_ratio * len(id_list))]
idstest = ids[round(train_valtest_ratio * len(id_list)):]
id_list_train = [id_list[i] for i in idstrain]
id_list_test = [id_list[i] for i in idstest]
#id_list_val = id_list[ids[round(train_valtest_ratio * len(id_list)): round(val_test_ratio + train_valtest_ratio)*len(id_list)]]
#id_list_test = id_list[ids[round(val_test_ratio + train_valtest_ratio)*len(id_list):]]


In [7]:
id_list_train[0]

{'id': 630449,
 'birdname': 'b14r16',
 'filepath': '/media/songbird/datapartition/mdgan_training_input_with_age_HDF/b14r16',
 'within_file': '/2011-06-22/b14r16U-f00547-2_0_52',
 'age_weight': 0.48,
 'duration': 473}

In [8]:
len(idstest)

160838

In [9]:
with open('/media/songbird/datapartition/id_list_train.pkl', 'wb') as f: 
    pickle.dump(id_list_train, f, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open('/media/songbird/datapartition/id_list_test.pkl', 'wb') as f: 
    pickle.dump(id_list_test, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
ages_train = [age]

# make a pytorch dataset for your dataset

In [23]:
from torch.utils import data

In [19]:
def transform(im):
    """
    This function should be used to transform data into the desired format for the network.
    inverse transoform should be an inverse of this function
    """
    im = from_polar(im)
    im, phase = lc.magphase(im)
    im = np.log1p(im)
    return im

In [39]:
class songbird_dataset(data.Dataset):
    def __init__(self, path2idlist, imageW):
        with open(path2idlist, 'rb') as f:
            self.id_list = pickle.load(f)
            self.imageW = imageW
        
    def __len__(self):
        # total number of samples
        return len(self.id_list)
    
    def __getitem__(self, index):
        # load one wav file and get a sample chunk from it
        ID = self.id_list[index]
        age_weight = ID['age_weight']
        # this 'ID' is a dictionary containing several fields,
        # use field 'filepath' and 'within_file' to get data
        f = h5py.File(ID['filepath'], 'r') 
        X = np.array(f.get(ID['within_file']))
        f.close()
        # transform 
        X = self.crop_and_transform(X)
        return torch.from_numpy(X), torch.Tensor([age_weight])
    
    def crop_and_transform(self, X):
        X = random_crop(X, width=self.imageW)
        X = transform(X)
        return X

In [62]:
import torch

In [46]:
songdataset = songbird_dataset('/mnt/Data/id_list_train.pkl', imageW=16)
songdataloader = data.DataLoader(songdataset, batch_size=32, shuffle=True, num_workers=6)

In [41]:
x, a = songdataset.__getitem__(1)

In [44]:
a

tensor([ 0.7700])

In [ ]:
for _ in range(2):
    for i,(local_batch, local_age) in enumerate(songdataloader):
        if i%100==0:
            print(local_batch.size())
            print(local_age.shape)